In [0]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving rebalance.csv to rebalance.csv


In [0]:
track_table_1 = pd.read_csv("50k.csv").iloc[:,1:]
track_table_2 = pd.read_csv("rebalance.csv").iloc[:,1:]# track_table = combined table
# remove duplicates
track_table = pd.concat([track_table_1, track_table_2], axis=0)
track_table.drop_duplicates(inplace=True)
track_table = track_table[['track_name', 'track_id','track_year', #'track_spotify_genre',
                           'art_name','art_id', 'alb_name','alb_id', 'art_genre',
                           'track_pop','duration_ms','time_signature', 'key', 'loudness','energy', 'speechiness',
                           'acousticness','mode','tempo', 'valence', 'danceability', 'instrumentalness',
                           'liveness']]

In [0]:
# ARTIST GENRE ANALYSIS
# track_table.dropna(inplace=True)
# Tranform hip hop into hip-hop, hip pop into hip-pop, early music into early-music
track_table['art_genre'] = track_table['art_genre'].str.replace('hip hop','hip-hop')
track_table['art_genre'] = track_table['art_genre'].str.replace('hip pop','hip-pop')
track_table['art_genre'] = track_table['art_genre'].str.replace('early music','early-music')

# create dictionary of word counts for each track (track_table['genre_words'])
# AND dictionary of word counts across all tracks (all_words)



import ast
genre_words = []
all_words = dict()

for genre_list in track_table['art_genre']: # for every artist
  artist_count = dict()
  genre_list = ast.literal_eval(genre_list)
  for genre_string in genre_list: # for each genre in artist_genre list
    for word in genre_string.split(): # for each word in each genre
      #Add to master list
      if word in all_words:
        all_words[word] += 1
      else:
        all_words[word] = 1
      #Add to artist-specific list
      if word in artist_count:
        artist_count[word] += 1
      else:
        artist_count[word] = 1
  genre_words.append(artist_count)
track_table['genre_words'] = genre_words

In [0]:
# Generate set of 10 most popular genres (popular_genres)
from collections import Counter 
all_words_count = Counter(all_words) 
popular_genres = set([word[0] for word in all_words_count.most_common(12)])

popular_genres.remove('rap')
popular_genres.remove('trap')
popular_genres.remove('modern')
popular_genres.remove('dance')

print("Most Popular Genres: ", popular_genres)

Most Popular Genres:  {'r&b', 'pop', 'indie', 'alternative', 'hip-hop', 'rock', 'house', 'country'}


In [0]:
# Add column (track_table["master_popular_genre"]) to each track, representing which of the primary genres the track falls into
popular_word_count = []
for words in track_table['genre_words']: # for every artist
  popular_words = words.copy()
  for word in words:
    if word not in popular_genres:
      del popular_words[word]
  popular_word_count.append(popular_words)

most_popular_master_genre = [Counter(song).most_common(1) for song in popular_word_count]
for item in range(0,len(most_popular_master_genre)):
  if len(most_popular_master_genre[item]) == 0:
    most_popular_master_genre[item] = np.nan
  else:
    most_popular_master_genre[item] = most_popular_master_genre[item][0][0]
track_table["master_popular_genre"] = most_popular_master_genre

In [0]:
print("Number of tracks that do not fall under top 10 genres: ", track_table[track_table["master_popular_genre"].isna()].count().track_id)
print(display(track_table.groupby("master_popular_genre").count().track_id))

Number of tracks that do not fall under top 10 genres:  16158


master_popular_genre
alternative     4545
country         7238
hip-hop         8146
house           5775
indie           5741
pop            17987
r&b             3678
rock            5298
Name: track_id, dtype: int64

None


In [0]:
track_table.to_csv('cleaned_tracks_genre.csv', index=False)
files.download('cleaned_tracks_genre.csv')

In [0]:
# pop_tracks = track_table[track_table.master_popular_genre == 'pop']
# track_table_popless = track_table[track_table.master_popular_genre != 'pop']
# pop_tracks_sampled = pop_tracks.sample(n=7000)
# track_table_balanced = pd.concat([track_table_popless, pop_tracks_sampled], axis=0)

# track_table_balanced.to_csv('cleaned_tracks_genre_balanced.csv', index=False)
# files.download('cleaned_tracks_genre_balanced.csv')

In [0]:
track_table.head()

,track_name,track_id,track_year,art_name,art_id,alb_name,alb_id,art_genre,track_pop,duration_ms,time_signature,key,loudness,energy,speechiness,acousticness,mode,tempo,valence,danceability,instrumentalness,liveness,genre_words,master_popular_genre
0,Tongue Tied,0GO8y8jQk1PkHzS31d699N,2011,Grouplove,3kVUvbeRdcrqQ3oHk5hPdx,Never Trust A Happy Song,3oylWMc9TTC6Nx4I6U3axc,"['indie pop', 'indie rock', 'la indie', 'moder...",77,218013.0,4.0,3.0,-5.835,0.936,0.0439,0.00847,1.0,112.960,0.371,0.560,0.000000,0.1610,"{'indie': 3, 'pop': 1, 'rock': 4, 'la': 1, 'mo...",rock
1,Pumped Up Kicks,7w87IxuO7BDcJ3YUqCyMTT,2011,Foster The People,7gP3bB2nilZXLfPHJhMdvc,Torches,7Kmmw7Z5D2UD5MVwdm10sT,"['indie pop', 'indietronica', 'modern rock', '...",82,239600.0,4.0,5.0,-5.849,0.710,0.0292,0.14500,0.0,127.975,0.965,0.733,0.115000,0.0956,"{'indie': 1, 'pop': 1, 'indietronica': 1, 'mod...",rock
2,Work Out,2wAJTrFhCnQyNSD3oUgTZO,2011,J. Cole,6l3HvQ5sa6mXTsMTB19rO5,Cole World: The Sideline Story,0fhmJYVhW0e4i33pCLPA5i,"['conscious hip-hop', 'hip-hop', 'nc hip-hop',...",77,235320.0,4.0,2.0,-6.903,0.427,0.1060,0.05310,1.0,93.004,0.216,0.831,0.000000,0.3070,"{'conscious': 1, 'hip-hop': 3, 'nc': 1, 'pop':...",hip-hop
3,Someone Like You,4kflIGfjdZJW4ot2ioixTB,2011,Adele,4dpARuHxo51G3z768sgnrY,21,1azUkThwd2HfUDdeNeT147,"['british soul', 'pop', 'uk pop']",77,285040.0,4.0,9.0,-8.251,0.330,0.0285,0.89200,1.0,135.109,0.285,0.559,0.000000,0.0975,"{'british': 1, 'soul': 1, 'pop': 2, 'uk': 1}",pop
4,Somebody That I Used To Know,4wCmqSrbyCgxEXROQE6vtV,2011,Gotye,2AsusXITU8P25dlRNhcAbG,Making Mirrors,1HjSyGjmLNjRAKgT9t1cna,['australian pop'],74,244973.0,4.0,0.0,-7.036,0.495,0.0370,0.59100,1.0,129.062,0.720,0.864,0.000133,0.0992,"{'australian': 1, 'pop': 1}",pop
